In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import imageio
import os

#Parameter combinations to test:
    #α=0,    N²=1e-6, M²=1e-6  Control (no strain) 
    #α=1e-6, N²=1e-6, M²=1e-6  Weak strain 
    #α=1e-5, N²=1e-6, M²=1e-6 Moderate strain (paper-like) 
    #α=1e-5, N²=1e-5, M²=1e-6 Strong stratification done
    #α=1e-5, N²=1e-6, M²=1e-5 Weaker front
    #α=1e-5, N²= 0 , M²=1e-6  Zero PV case to compare with ST13
    #α=1e-5, N²= 1e-3 , M²=1e-6  Explodes!
    
    #α=0, N²= 0 , M²=9e-6  B00 case


# File naming convention: filename = "front_α=(α)_N²=(N²)_M²=(M²).nc"
ds = xr.open_dataset('run_a=0_N^2=1e-6_M^2=1e-6.nc', engine = 'netcdf4', decode_times=False)

In [25]:
ds.b

<xarray.DataArray 'b' (time: 41, z_aac: 20, x_caa: 240)> Size: 787kB
[196800 values with dtype=float32]
Coordinates:
  * time     (time) float64 328B 0.0 600.0 1.2e+03 ... 2.28e+04 2.34e+04 2.4e+04
  * x_caa    (x_caa) float32 960B -2.49e+05 -2.469e+05 ... 2.469e+05 2.49e+05
  * z_aac    (z_aac) float32 80B -975.0 -925.0 -875.0 ... -125.0 -75.0 -25.0
Attributes:
    units:      m/s²
    long_name:  Buoyancy

## Buoyancy Animation

In [52]:
import xarray as xr
import matplotlib.pyplot as plt
import imageio
import os

# === Load dataset ===
ds = xr.open_dataset('larger_domain2.nc', decode_times=False)
b = ds['b']
x = ds['x_caa'].values / 1e3  # convert to km
z = ds['z_aac'].values
time_values = ds['time'].values

# === Prepare a folder for frames ===
os.makedirs("frames", exist_ok=True)

# === Create frames ===
images = []
for i in range(len(time_values)):
    plt.figure(figsize=(8, 4))
    ax = plt.gca()

    # Get 2D buoyancy data at time i
    b_data = b.isel(time=i ).values

    # Filled color map
    pc = ax.pcolormesh(x, z, b_data, shading='auto', cmap='RdBu_r',
                       vmin=float(b.min()), vmax=float(b.max()))
    plt.colorbar(pc, ax=ax, label="b (m/s²)")

    # Contour lines
    contour_levels = 10  # or use a list like [-1e-4, -5e-5, 0, 5e-5, 1e-4]
    cs = ax.contour(x, z, b_data, levels=contour_levels, colors='black', linewidths=0.5)
    #ax.clabel(cs, inline=True, fontsize=6, fmt="%.0e")

    # Title and labels
    plt.title(f"Time = {time_values[i]/86400:.2f} days")
    ax.set_xlabel("x (km)")
    ax.set_ylabel("z (m)")
    ax.set_xlim(-100, 100)

    # Save frame
    frame_path = f"frames/frame_{i:04d}.png"
    plt.savefig(frame_path, dpi=150)
    images.append(imageio.imread(frame_path))
    plt.close()

# === Save as animation ===
imageio.mimsave('buoyancy_animation.mp4', images, fps=5)

print("✅ Animation saved as 'buoyancy_animation.mp4' with buoyancy contours.")


C:\Users\Tom Cummings\AppData\Local\Temp\ipykernel_22504\2829249627.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(frame_path))
C:\Users\Tom Cummings\AppData\Local\Temp\ipykernel_22504\2829249627.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(frame_path))
C:\Users\Tom Cummings\AppData\Local\Temp\ipykernel_22504\2829249627.py:44: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import i

✅ Animation saved as 'buoyancy_animation.mp4' with buoyancy contours.


In [ ]:
ds.b[:20,10,100:140].plot(vmin = -0.01, vmax = 0.01)